
# RoamRhythm — FIT → GPX → POIs → SSML (Interactive Notebook)

This notebook provides an **interactive**, well-documented workflow to:
- Convert **`.FIT` → `.GPX`**
- Generate **evenly spaced POIs** over distance
- Produce **calm/enthusiastic SSML** (intro, per-POI, outro)

> **Stack**: Conda (Python 3.13.5), `fitdecode` or `fitparse`, `gpxpy`, `numpy`  
> **Workflow**: iterate here → migrate stable logic back to a Python CLI/script later.



## 0) Environment & dependencies (Conda + pip)

Create the environment once, then run Jupyter from it:

```bash
conda create -n roamrhythm python=3.13.5 -y
conda activate roamrhythm
pip install --upgrade pip
pip install jupyterlab nbformat fitdecode fitparse gpxpy numpy
```
> You only need **one** of `fitdecode` or `fitparse`. Installing both increases compatibility.



## 1) Project config (auto locate your FIT file)

- Set `FIT_HINT` to either an **absolute path** or just the **filename**.  
- If only a filename is given, we **auto-search** recursively from the notebook folder.


In [4]:

from pathlib import Path
import sys

# --- Configure either a direct path OR just a filename to auto-search ---
FIT_HINT = r".\Data\Test10K_BE_UD.fit"   # change to your file name or absolute path
OUTDIR   = Path("./out")
PACE_KMH = 4.0
POI_COUNT = 10
USE_FITDECODE = True  # set False to try fitparse

def resolve_fit_path(hint: str) -> Path:
    p = Path(hint)
    if p.exists():
        return p.resolve()
    here = Path.cwd()
    candidate = here / hint
    if candidate.exists():
        return candidate.resolve()
    matches = list(here.rglob(hint if "*" in hint else f"*{Path(hint).stem}*.fit"))
    if matches:
        return matches[0].resolve()
    raise FileNotFoundError(
        f"FIT file not found.\n"
        f"Tried: {p}\n"
        f"CWD : {here}\n"
        f"Tips:\n"
        f" - Put the .fit next to the notebook OR give an absolute path.\n"
        f" - On Windows, use raw strings: r\"C:\\path\\to\\file.fit\".\n"
        f" - Check extension case (.fit vs .FIT).\n"
    )

FIT_PATH = resolve_fit_path(FIT_HINT)
OUTDIR.mkdir(parents=True, exist_ok=True)
print("Using FIT:", FIT_PATH)
print("Output dir:", OUTDIR.resolve())


Using FIT: C:\Development\Python\RoamRhythm\data\Test10K_BE_UD.fit
Output dir: C:\Development\Python\RoamRhythm\out



## 2) Imports & helpers


In [2]:
%pip install nbformat fitdecode fitparse gpxpy numpy

Defaulting to user installation because normal site-packages is not writeable
  Using cached fitdecode-0.11.0-py3-none-any.whl.metadata (11 kB)
  Using cached fitparse-1.2.0.tar.gz (65 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached gpxpy-1.6.2-py3-none-any.whl.metadata (5.9 kB)
Using cached fitdecode-0.11.0-py3-none-any.whl (109 kB)
Using cached gpxpy-1.6.2-py3-none-any.whl (42 kB)
  Created wheel for fitparse: filename=fitparse-1.2.0-py3-none-any.whl size=68236 sha256=bd836a35f334cfb5632198f2f5ac56b9de02484b84e500705b250827ef248528
  Stored in directory: c:\users\charl\appdata\local\pip\cache\wheels\6a\8a\17\d5e85db99c2efece41b1aa8388d9e5c42d1796021b30f3f1ee
Successfully built fitparse

   ---------------------------------------- 0/3 [fitparse]
   ------------- -------------------------- 1/3 [gpxpy]
   -------------------------- ------------- 2/3 [fitdecode]
   -------------------------- ------------- 2/3 [fitde

  DEPRECATION: Building 'fitparse' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'fitparse'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [7]:

import math, csv, json
from datetime import datetime, timedelta, timezone
from typing import List, Tuple, Optional

import numpy as np
import gpxpy, gpxpy.gpx

# Try selected FIT backend first; fall back to the other
fit_backend = None
if USE_FITDECODE:
    try:
        import fitdecode  # type: ignore
        fit_backend = "fitdecode"
    except Exception as e:
        print("fitdecode not available:", e)
        fit_backend = None

if fit_backend is None:
    try:
        from fitparse import FitFile  # type: ignore
        fit_backend = "fitparse"
    except Exception as e:
        print("fitparse not available:", e)
        raise RuntimeError("No FIT backend available. Please install fitdecode or fitparse.")

def haversine(lat1, lon1, lat2, lon2):
    R = 6371000.0
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R * c  # meters

def cumulative_distance(points: List[Tuple[float,float]]) -> List[float]:
    dists = [0.0]
    for i in range(1, len(points)):
        d = haversine(points[i-1][0], points[i-1][1], points[i][0], points[i][1])
        dists.append(dists[-1] + d)
    return dists

def read_fit_fitdecode(path: Path):
    lats, lons, eles, times = [], [], [], []
    with fitdecode.FitReader(str(path)) as rd:
        for frame in rd:
            if isinstance(frame, fitdecode.records.FitDataMessage) and frame.name == 'record':
                lat = frame.get_value('position_lat')
                lon = frame.get_value('position_long')
                ele = frame.get_value('enhanced_altitude')
                t = frame.get_value('timestamp')
                def semi_to_deg(x): return x * (180.0 / 2**31) if x is not None else None
                lat = semi_to_deg(lat); lon = semi_to_deg(lon)
                if lat is not None and lon is not None:
                    lats.append(lat); lons.append(lon); eles.append(ele if ele is not None else 0.0)
                    times.append(t if isinstance(t, datetime) else None)
    return lats, lons, eles, times

def read_fit_fitparse(path: Path):
    lats, lons, eles, times = [], [], [], []
    ff = FitFile(str(path))
    for record in ff.get_messages('record'):
        vals = {d.name: d.value for d in record}
        lat = vals.get('position_lat'); lon = vals.get('position_long')
        ele = vals.get('enhanced_altitude', 0.0); t = vals.get('timestamp', None)
        def semi_to_deg(x): return x * (180.0 / 2**31) if x is not None else None
        lat = semi_to_deg(lat); lon = semi_to_deg(lon)
        if lat is not None and lon is not None:
            lats.append(lat); lons.append(lon); eles.append(ele if ele is not None else 0.0)
            times.append(t if isinstance(t, datetime) else None)
    return lats, lons, eles, times

def fit_to_track(path: Path):
    if fit_backend == "fitdecode":
        return read_fit_fitdecode(path)
    elif fit_backend == "fitparse":
        return read_fit_fitparse(path)
    else:
        raise RuntimeError("No FIT backend selected.")

def make_gpx(lats, lons, eles, times, creator="RoamRhythm-notebook"):
    gpx = gpxpy.gpx.GPX()
    gpx.creator = creator
    trk = gpxpy.gpx.GPXTrack()
    gpx.tracks.append(trk)
    seg = gpxpy.gpx.GPXTrackSegment()
    trk.segments.append(seg)
    for lat, lon, ele, t in zip(lats, lons, eles, times):
        seg.points.append(gpxpy.gpx.GPXTrackPoint(latitude=lat, longitude=lon, elevation=ele, time=t))
    return gpx

def interpolate_time_from_pace(start_time: Optional[datetime], dist_m: float, pace_kmh: float) -> datetime:
    if start_time is None:
        start_time = datetime(1970,1,1, tzinfo=timezone.utc)
    hours = dist_m / (pace_kmh * 1000.0)
    return start_time + timedelta(hours=hours)

def ssml_wrap(text: str) -> str:
    return f'<speak><prosody rate="85%" pitch="+0st" volume="+0.0dB">{text}</prosody></speak>'

POI_TITLES = [
    "Landschap & waterstaat","Ontstaan van nederzettingen","Ambachten en handel","Architectuur in de regio",
    "Natuur & biodiversiteit","Polders, dijken en gemalen","Oude routes & verbindingen","Oorlogssporen en herdenking",
    "Bestuur & gemeentefusies","Moderne tijd & toekomst"
]
POI_TEMPLATES = [
    "Dit landschap is gevormd door water en wind. Let op sloten, kades en hoogtes: ze vertellen hoe mensen het water sturen. Een klein weetje: veel polderwegen liggen net hoger dan omliggende velden.",
    "Veel dorpen ontstonden bij kruisingen van wegen en water. Luister hoe handel en ambacht bewoners trokken—vaak rond een kerk of plein als hart van de gemeenschap.",
    "Ambachten bloeiden waar grondstoffen en routes elkaar raken. Denk aan molenaars, smeden, scheepstimmerlieden of linnenwevers—ieder gebonden aan seizoen en plaats.",
    "Kijk naar metselwerk, dakvormen en gevelopeningen. Baksteen en nuchtere lijnen domineren vaak; wederopbouw en modernisering lieten ook hun sporen na.",
    "Let op vogels aan randen van water en riet. Fluisterstil bewegen vaak waterhoentjes; in sloten zie je soms libellen. Tip: loop even langzamer en luister naar lagen van geluid.",
    "Polderen is samenwerken: dijken houden water buiten, sloten voeren af, gemalen regelen peil. Een metafoor: het landschap ademt—in natte tijden trager, in droge sneller.",
    "Routes vormen ritme: karrenpaden, trekvaarten, kanalen of spoorlijnen verplaatsen mensen en ideeën. Oude tracés wonen soms voort als stille fietspaden.",
    "Sporen van oorlog vragen bedachtzaamheid. Monumenten zijn ankerpunten van herinnering; ze nodigen uit tot stilstaan, kijken, en zachtjes verdergaan.",
    "Gemeenten veranderden mee met tijd en taken. Fusies bundelden diensten; grenzen verschoof men om samen sterker te staan in zorg, onderwijs en beheer.",
    "Vandaag draait veel om balans: natuurontwikkeling, energie, wonen en recreatie. Kijk om je heen: vernieuwing en behoud kunnen elkaar versterken."
]



## 3) Load `.FIT` → write `.GPX`


In [8]:

lats, lons, eles, times = fit_to_track(FIT_PATH)
if len(lats) < 2:
    raise RuntimeError("Not enough track points in FIT.")
gpx = make_gpx(lats, lons, eles, times)
gpx_path = OUTDIR / (FIT_PATH.stem + ".gpx")
gpx_path.write_text(gpx.to_xml(), encoding="utf-8")
print(f"GPX written: {gpx_path.resolve()}")
print(f"Track points: {len(lats)}")


GPX written: C:\Development\Python\RoamRhythm\out\Test10K_BE_UD.gpx
Track points: 1196



## 4) Compute distance & build evenly-spaced POIs


In [9]:

points = list(zip(lats, lons))
cum = cumulative_distance(points)  # meters
total_m = cum[-1]
print(f"Total distance: {total_m/1000.0:.2f} km")

# targets from 9% to 99% of route (avoid overlap with intro/outro)
targets = np.linspace(total_m * 0.09, total_m * 0.99, POI_COUNT)

# find first timestamp (if any)
t0 = next((t for t in times if isinstance(t, datetime)), None)

pois = []
for i, tm in enumerate(targets):
    idx = int(np.argmin([abs(d - tm) for d in cum]))
    km = round(cum[idx]/1000.0, 2)
    lat, lon = float(lats[idx]), float(lons[idx])
    eta = times[idx] if (idx < len(times) and isinstance(times[idx], datetime)) else None
    if eta is None:
        eta = (t0 if t0 else datetime(1970,1,1, tzinfo=timezone.utc))
        hours = (cum[idx] / 1000.0) / PACE_KMH
        eta = eta + timedelta(hours=hours)
    dur_s = 100 if i % 2 == 0 else 95
    title = POI_TITLES[i % len(POI_TITLES)]
    text  = POI_TEMPLATES[i % len(POI_TEMPLATES)]
    ssml  = ssml_wrap(f"{title}. {text}")
    pois.append({
        "id": f"POI-{i+1:02d}",
        "km": km, "lat": lat, "lon": lon,
        "eta_utc": eta.astimezone(timezone.utc).isoformat(),
        "duration_s": dur_s, "title": title, "ssml": ssml
    })

print(f"POIs generated: {len(pois)}")


Total distance: 10.14 km
POIs generated: 10



## 5) Intro & Outro SSML


In [10]:

intro_text = ("Welkom bij deze wandeling. We nemen u in een rustig tempo mee door het landschap, "
              "met korte verhalen over de omgeving, afgewisseld met muziek uit de jaren tachtig, "
              "taalprikkels Noors en ontspannen stretchmomenten. "
              "Wandel op uw gemak, kijk om u heen en luister wanneer we een interessant punt passeren. "
              "Aan het begin starten we met een korte warming-up. Klaar? Dan gaan we op pad.")
outro_text = ("We zijn bijna aan het einde van de route. Neem even de tijd om na te voelen hoe uw lichaam is "
              "opgewarmd. Straks sluiten we af met enkele rustige stretches. "
              "Dank voor het meewandelen. Heeft u iets nieuws ontdekt of geleerd—een woord Noors, een stukje "
              "geschiedenis? Bewaar het gevoel van ruimte en ritme. Tot de volgende keer, en fijne dag verder.")
intro_ssml = ssml_wrap(intro_text)
outro_ssml = ssml_wrap(outro_text)



## 6) Write artifacts (GPX, POI CSV, SSML files, Manifest)


In [11]:

ssml_dir = OUTDIR / "ssml"
ssml_dir.mkdir(exist_ok=True)

# Save SSML
(ssml_dir / "intro.ssml").write_text(intro_ssml, encoding="utf-8")
(ssml_dir / "outro.ssml").write_text(outro_ssml, encoding="utf-8")
for poi in pois:
    (ssml_dir / f"{poi['id']}.ssml").write_text(poi["ssml"], encoding="utf-8")

# Save CSV
csv_path = OUTDIR / "pois.csv"
with csv_path.open("w", newline="", encoding="utf-8") as f:
    w = csv.writer(f)
    w.writerow(["id","km","lat","lon","eta_utc","duration_s","title","ssml_file"])
    for poi in pois:
        w.writerow([poi["id"], poi["km"], poi["lat"], poi["lon"], poi["eta_utc"], poi["duration_s"], poi["title"], f"ssml/{poi['id']}.ssml"])

# Save Manifest
manifest = {
    "route": {
        "meters": round(float(cum[-1]), 1),
        "km": round(float(cum[-1])/1000.0, 3)
    },
    "gpx": (OUTDIR / (FIT_PATH.stem + ".gpx")).name,
    "pace_kmh": PACE_KMH,
    "ssml": {
        "intro": "ssml/intro.ssml",
        "outro": "ssml/outro.ssml"
    },
    "pois": [
        {
            "id": p["id"], "km": p["km"], "lat": p["lat"], "lon": p["lon"],
            "eta_utc": p["eta_utc"], "duration_s": p["duration_s"],
            "title": p["title"], "ssml": f"ssml/{p['id']}.ssml"
        } for p in pois
    ]
}
manifest_path = OUTDIR / "manifest.json"
manifest_path.write_text(json.dumps(manifest, ensure_ascii=False, indent=2), encoding="utf-8")

print("Artifacts written:")
print(" - GPX:", gpx_path.resolve())
print(" - CSV:", csv_path.resolve())
print(" - SSML dir:", ssml_dir.resolve())
print(" - Manifest:", manifest_path.resolve())


Artifacts written:
 - GPX: C:\Development\Python\RoamRhythm\out\Test10K_BE_UD.gpx
 - CSV: C:\Development\Python\RoamRhythm\out\pois.csv
 - SSML dir: C:\Development\Python\RoamRhythm\out\ssml
 - Manifest: C:\Development\Python\RoamRhythm\out\manifest.json



## 7) Git & GitHub sync

```bash
git init
git add .
git commit -m "RoamRhythm: FIT→GPX→POI→SSML notebook"
git branch -M main
git remote add origin https://github.com/poco-irrilevante/RoamRhythm.git
git push -u origin main
```
> Tip: add `/out/` to `.gitignore` if you don't want to commit generated artifacts.



## 8) Migration back to Python script

When you're happy with the workflow, extract functions from cells 2–6 into a module (e.g., `roamrhythm/core.py`) and add a small CLI (argparse) to create a script-friendly tool.
